In [12]:
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import os
import json


def get_item(name):
    # link
    path = "output/" + name + ".html"
    
    # file
    file = open(path, "r", encoding="utf-8")

    # read file
    html = BeautifulSoup(file.read())
    
    # the content of the page
    content = html.find("div", {"id": "content"})

    # we want the Erwägung
    erw = content.find('h1', text="Erwägungen")

    data = {}

    # we take the paragraphs
    paragraphs = []
    
    while erw:
        if erw.find('strong') is not None:
            paragraph = {}
            
            # we want the original text
            paragraph["original"] = str(erw)
            
            # the index of the paragraph
            index = erw.find('strong').extract().text
            paragraph["index"] = index
            
            # we find the links
            links = erw.findChildren("a", {"class": "LexLink"})
            hrefs = []
            for link in links:
                hrefs.append(link["href"][1:])
                link.extract()
            paragraph["links"] = hrefs
            
            # we remove to get text
            thinsp = erw.findChildren("span", {"class": "thinsp"})
            for t in thinsp:
                t.extract()
            notes = erw.findChildren("span", {"class": "note"})
            for note in notes:
                note.extract()
            pagebk = erw.findChildren("span", {"class": "pagebreak"})
            for bk in pagebk:
                bk.extract()
            
            # we save only the text
            paragraph["content"] = erw.text
            
            # check that we have a big enough text
            if len(paragraph["content"]) > 10:
                paragraphs.append(paragraph)
                
    
        # next paragraph
        erw = erw.findNext("p")
    
    # save the paragraphs and return
    data["paragraphs"] = paragraphs
    return data


path = "entscheide.txt"
with open(path, "r") as file:
    names = file.read().splitlines()
    
output = {}
for name in names:
    output[name] = get_item(name)


with open('json_data.json', 'w', encoding="utf-8") as outfile:
    content = json.dumps(output, indent=4, ensure_ascii=False)
    outfile.write(content)